# Test for Models module

## Librarie

In [1]:
import os, sys

sys.path.insert(0, os.pardir)

import torch
from torch import nn

from src.models.models import UNet3d
from src.models.trainer import Trainer

## Model

In [2]:

# pool of non-square window
m = nn.MaxPool3d((8, 1, 1))
input = torch.randn(8, 1, 8, 256, 256)
output = m(input)

display(input.shape)
display(output.shape)

torch.Size([8, 1, 8, 256, 256])

torch.Size([8, 1, 1, 256, 256])

In [3]:
m = nn.Linear(20, 30)
input = torch.randn(128, 20)
output = m(input)
display(input.shape)
display(output.size())

torch.Size([128, 20])

torch.Size([128, 30])

In [4]:
input = torch.randn(32, 1, 5, 5)
display(input.shape)
# With default parameters
m = nn.Flatten()
output = m(input)
display(output.size())
# With non-default parameters
m = nn.Flatten(0, 2)
output = m(input)
display(output.size())

torch.Size([32, 1, 5, 5])

torch.Size([32, 25])

torch.Size([160, 5])

In [8]:
list_channels = [1, 32, 64, 128]
unet3d = UNet3d(list_channels, depth=8)

8


In [9]:
inputs = torch.randn(8, 1, 8, 256, 256)

In [10]:
unet3d(inputs).shape

torch.Size([8, 1, 8, 256, 256])


torch.Size([8, 1, 1, 256, 256])